# Data Cleaning/Wrangling

Author: Gillian A. McGinnis, final-semester M.S. Information Science - Machine Learning  
The University of Arizona College of Information  
INFO 698 - Capstone  
Start date: 24 September 2025  
Last updated: 19 November 2025

In [1]:
"""
Module providing supporting code for preparing data for EDA and analysis.
"""

'\nModule providing supporting code for preparing data for EDA and analysis.\n'

## Load Required Libraries

In [2]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For time series data management
# import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import datetime as dt
# import pyarrow as pa

# For data importing
import os

In [3]:
## (Optional chunk)
# Current session information
import session_info
session_info.show(dependencies=False)

## Load data

Files of interest:
- `weir_calibration.csv` includes calibration points for the weir
- `bci_lutzweir_combined.csv` includes raw runoff measurement, corrected runoff measurement, data source (*Chart measurements can be removed)
- `bci_cl_ra_elect2.CSV` has corrected rainfall (`ra`) in mm with measurements of `0` as `NA`s (`bci_cl_ra_elect.csv` has `0`s)
- `bci_lutz_deep_gsm_man.csv`, `bci_lutz_shallow_gsm_man.csv` have soil moisture measurements (water by wet weight and water by dry weight; one can be chosen for analysis as they are linearly related)
<!-- `bci_cl_ra_elect.csv` has corrected rainfall (`ra`) in mm, contains `0`s (large file) -->

All values level values are in mm, and datetime is in UTC-5 (Panama time zone).

### Import

In [4]:
# The data is located in a different folder from the cwd
def get_data_path(input_filename):
    """Get the file path for specified data file.

    Args:
        input_filename (str): Data file name in the ~/data folder.

    Returns:
        str: Full absolute path to data file.
    """
    path_data = os.path.abspath(os.path.join('..', 'data', input_filename))
    return path_data

#### Weir calibrations

In [5]:
## Calibrations dataset
data_raw_calibration = pd.read_csv(
    # Location of the dataset in the repo
    # "data/weir_calibration.csv",
    get_data_path("weir_calibration.csv"),
    # Specify columns to load
    ## note- weir_hour is a repeat of the time in datetime and can be skipped
    usecols = ['datetime', 'weir_level'],
    # Convert datetime stamp strings to datetime objects
    parse_dates = ['datetime'],
    # Specify the types for specific columns
    dtype = {
        'weir_level': 'Int8'
    },
    # Specify the string formatting of the datetime stamps
    date_format = "%d/%m/%Y %H:%M:%S",
    # Use datetime stamp as index
    index_col = 'datetime'
)

# Arrange chronologically
data_raw_calibration = data_raw_calibration.sort_index()

data_raw_calibration.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6465 entries, 1994-01-03 08:46:00 to 2025-09-02 08:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   weir_level  6465 non-null   Int8 
dtypes: Int8(1)
memory usage: 63.1 KB


#### Weir measurements

In [6]:
# Combined data

data_raw_weir = pd.read_csv(
    # Location of the dataset in the repo
    # "data/bci_lutzweir_combined.csv",
    get_data_path("bci_lutzweir_combined.csv"),
    # Specify columns to load
    usecols = ['datetime', 'level', 'raw', 'chk_note', 'chk_fail', 'comment', 'source'],
    # Specify the types for specific columns
    dtype = {
        'level':'float32',
        'raw':'float32',
        'chk_note':'category',
        'chk_fail':'category',
        'comment':'category',
        'source':'category'
    },
    # Convert datetime stamp strings to datetime objects
    parse_dates = ['datetime'],
    # Specify the string formatting of the datetime stamps
    date_format = "%d/%m/%Y %H:%M:%S",
    # Use datetime stamp as index
    index_col = 'datetime'
)

## This variation checks first if the dataset is already loaded into the workspace
# try:
#     if data_weir.empty == False:
#         print("Data loaded, random sample shown below")
#         print(data_weir.sample(n=5))
# except NameError:
#     print("Data has not yet been read in, loading now...")
#     data_weir = pd.read_csv(
#         "data/bci_lutzweir_combined.csv",
#         usecols = ['datetime', 'level', 'raw', 'chk_note', 'chk_fail', 'comment', 'source'],
#         parse_dates=['datetime'],
#         dtype = {'source':'category', 'chk_note':'category', 'chk_fail':'str', 'comment':'str'},
#         date_format='%d/%m/%Y %H:%M:%S'
#     )

# Arrange chronologically
data_raw_weir = data_raw_weir.sort_index()

data_raw_weir.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3951119 entries, 1972-01-01 01:00:00 to 2025-08-01 13:00:00
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   level     float32 
 1   raw       float32 
 2   chk_note  category
 3   chk_fail  category
 4   comment   category
 5   source    category
dtypes: category(4), float32(2)
memory usage: 75.4 MB


#### Rainfall

In [7]:
# Rainfall dataset

# This data set skips the 0 readings (therefore much smaller):
data_raw_rainfall = pd.read_csv(
    # Location of the dataset in the repo
    # "data/bci_elect_cl_ra/bci_cl_ra_elect2.CSV",
    get_data_path("bci_elect_cl_ra/bci_cl_ra_elect2.CSV"),
    # Specify the types for specific columns
    dtype = {
        'ra':'float32',
        'raw':'float32',
        'chk_note':'category',
        'chk_fail':'category'
    },
    # Convert datetime stamp strings to datetime objects
    parse_dates = ['datetime'],
    # Specify the string formatting of the datetime stamps
    date_format = "%d/%m/%Y %H:%M:%S",
    # Use datetime stamp as index
    index_col = 'datetime'
)

# Arrange chronologically
data_raw_rainfall = data_raw_rainfall.sort_index()

# # This data set includes the 0 readings:
# data_raw_rainfall_zeroes = pd.read_csv(
#         "data/bci_elect_cl_ra/bci_cl_ra_elect.csv",
#         usecols = ['datetime', 'ra', 'raw', 'chk_note', 'chk_fail'],
#         # "data/bci_elect_cl_ra/bci_cl_ra_elect2.CSV",
#         # usecols = ['datetime', 'level', 'raw', 'chk_note', 'chk_fail', 'comment', 'source'],
#         parse_dates=['datetime'],
#         dtype = {'chk_note':'category', 'chk_fail':'str'},
#         # dtype = {'source':'category', 'chk_note':'category', 'chk_fail':'str', 'comment':'str'},
#         date_format='%d/%m/%Y %H:%M:%S'
#     )
# # # Arrange chronologically
# data_raw_rainfall_zeroes = data_raw_rainfall_zeroes.sort_index()

data_raw_rainfall.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 179640 entries, 1929-01-02 08:00:00 to 2025-08-04 11:55:00
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   ra        179640 non-null  float32 
 1   raw       179640 non-null  float32 
 2   chk_note  179640 non-null  category
 3   chk_fail  29 non-null      category
dtypes: category(2), float32(2)
memory usage: 3.1 MB


#### Soil moisture

*A note about the soil datasets:

Both `h2o_by_wet` and `h2o_by_dry` are available in the datasets.
Because they are linearly related to each other, only one of them is necessary for modelling.
Arbitrarily, `h2o_by_wet` has been chosen for this analysis.

In [8]:
# Soil datasets

# Shallow
data_raw_soil_shallow = pd.read_csv(
    # Location of the dataset in the repo
    # "data/bci_manual_soilh/bci_lutz_shallow_gsm_man.csv",
    get_data_path("bci_manual_soilh/bci_lutz_shallow_gsm_man.csv"),
    # Specify columns to load
    usecols = ['date', 'depth', 'sample', 'h2o_by_wet', 'chk_note', 'chk_fail'],
    # Specify the types for specific columns
    dtype = {
        'h2o_by_wet':'float32',
        'depth':'category',
        'sample':'category',
        'chk_note':'category',
        'chk_fail':'category'
    },
    # Convert date stamp strings to date objects
    parse_dates = ['date'],
    # Specify the string formatting of the date stamps
    date_format = "%d/%m/%Y",
    # Use date stamp as index
    index_col = 'date'
)

# Deep
data_raw_soil_deep = pd.read_csv(
    # Location of the dataset in the repo
    # "data/bci_manual_soilh/bci_lutz_deep_gsm_man.csv",
    get_data_path("bci_manual_soilh/bci_lutz_deep_gsm_man.csv"),
    # Specify columns to load
    usecols = ['date', 'depth', 'sample', 'h2o_by_wet', 'chk_note', 'chk_fail'],
    # Specify the types for specific columns
    dtype = {
        'h2o_by_wet':'float32',
        'depth':'category',
        'sample':'category',
        'chk_note':'category',
        'chk_fail':'category'
    },
    # Convert date stamp strings to date objects
    parse_dates = ['date'],
    # Specify the string formatting of the date stamps
    date_format = "%d/%m/%Y",
    # Use date stamp as index
    index_col = 'date'
)

# Arrange chronologically
data_raw_soil_shallow = data_raw_soil_shallow.sort_index()
data_raw_soil_deep = data_raw_soil_deep.sort_index()

data_raw_soil_shallow.info()
data_raw_soil_deep.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18556 entries, 1972-03-03 to 2025-06-26
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   depth       18556 non-null  category
 1   sample      18556 non-null  category
 2   h2o_by_wet  18556 non-null  float32 
 3   chk_note    18556 non-null  category
 4   chk_fail    178 non-null    category
dtypes: category(4), float32(1)
memory usage: 291.5 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15637 entries, 1972-03-03 to 2025-06-26
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   depth       15637 non-null  category
 1   sample      15637 non-null  category
 2   h2o_by_wet  15637 non-null  float32 
 3   chk_note    15637 non-null  category
 4   chk_fail    20 non-null     category
dtypes: category(4), float32(1)
memory usage: 245.4 KB


## Clean

Data cleanup is necessary to ensure ease of uniting the sets, conducting a test/train split, and creation of & fitting of the models.

### Dates

In [9]:
# Explore: Get earliest and latest dates of sources

cat_source = data_raw_weir['source'].unique().tolist()
# Header for printed table
print("Earliest", "\t    ", "Latest", "\t\t", "Source")
# Iterate across each source type
for cat in cat_source:
    # If the source is NaN
    if pd.isna(cat) == True:
        temp_subset = data_raw_weir[data_raw_weir['source'].isnull()]
    else:
        temp_subset = data_raw_weir[data_raw_weir['source'] == cat]
    # Sort index
    temp_subset = temp_subset
    # Print
    print(temp_subset.index[0], "", temp_subset.index[-1], cat)

# Save space, remove no longer needed items
del cat_source, cat, temp_subset

Earliest 	     Latest 		 Source
1972-01-01 01:00:00  2015-03-18 14:15:00 CHART
1972-09-16 00:15:00  2025-08-01 13:00:00 nan
1989-07-19 11:55:00  1996-10-01 23:55:00 CHART+AF
1996-10-02 00:00:00  2013-01-13 05:50:00 ISCO
2012-04-23 08:30:00  2012-04-24 08:35:00 ESTIMATED
2014-08-22 10:30:00  2021-05-19 09:40:00 RADAR
2018-08-31 10:05:00  2018-09-05 12:55:00 TROLL


#### CHART removal

Only values that are not solely reliant on CHART (i.e., after 1989) will be evaluated.

In [10]:
# Filter the dataset to start once values stopped by being recorded by CHART
date_weir_start = data_raw_weir[
    # Remove CHART values
    (data_raw_weir['source'] != 'CHART') &
    # and
    # Remove values without indicated source
    (~data_raw_weir['source'].isnull())
    # Pull earliest timestamp
    ].index[0]
# Get latest data point timestamp
date_weir_end = data_raw_weir.index[-1]

# Sanity check: it is expected that the start timestamp will be CHART+AF source
if date_weir_start != data_raw_weir[data_raw_weir['source'] == 'CHART+AF'].index[0]:
    print("-----!! Warning: Check start date !!-----",
          "Calculated:\t", date_weir_start, "\n"
          "Actual:\t\t", data_raw_weir[data_raw_weir['source'] == 'CHART+AF'].index[0], "\n")

print("Non- CHART-only values:", date_weir_start, "through", date_weir_end)

Non- CHART-only values: 1989-07-19 11:55:00 through 2025-08-01 13:00:00


#### 2-Year Failure

The ISCO sensor failed in early 2013, and there was no backup.
Values were recording using the CHART resource, and gap filled accordingly.
Electronic recording resumed with RADAR in late 2014.

The model cannot be trained on this gap of data, as it is using `CHART` values, and all `raw` values report `-999.0`.

In [11]:
# data_gap = data_raw_weir['2013-01-01 00:00:00':'2014-08-22 23:59:59']
# Isolate the rough start and stop dates of the gap
data_gap = data_raw_weir['2013-01-13 05:00:00':'2014-08-22 23:59:59'].copy()

# Get the earliest date of gap filling
date_gap_start = data_gap[data_gap['source'] == 'CHART'].index[0]

# Get the latest date of gap filling
date_gap_end = data_gap[data_gap['source'] != 'RADAR'].index[-1]

print("Two year gap:", date_gap_start, "through", date_gap_end)
## OLD EXPECTED -- 2013-01-02 18:54:38 through 2014-08-22 10:21:32
## ADJ EXPECTED -- 2013-01-13 05:54:01 through 2014-08-22 10:21:32

# Cleanup
del data_gap

Two year gap: 2013-01-13 05:54:01 through 2014-08-22 10:21:32


#### Applying

Removing larger gaps of irrelevant data help with memory and feature engineering later.

In [12]:
# Create function to filter dates
def filter_dates(input_dataset, input_date_start, input_date_end, drop_dates = False):
# def filter_dates(input_dataset, input_date_start = date_weir_start, input_date_end = date_weir_end, drop_dates = False):
    """Filter data set to specified start and end dates.
    
    Args:
        input_dataset (pd.DataFrame): Data indexed by datetime.
        input_date_start (timestamp): The start date, defaults to the earliest from the combined data set.
        input_date_end (timestamp): The end date, defaults to the earliest from the combined data set.
        drop_dates (bool): Whether to remove the values between the specified dates.
    
    Returns:
        pd.DataFrame: Sorted and filtered dataset to or without the specified range.
    """
    # Sort the dataframe
    # data_filtered = input_dataset.copy().sort_index()
    data_filtered = input_dataset.sort_index()
    # Filter to select inputted dates
    if drop_dates == False:
        # data_subset = data_subset.loc[input_date_start:input_date_end]
        data_filtered = data_filtered[input_date_start:input_date_end]
    # Remove data between inputted dates
    else:
        data_filtered = data_filtered.drop(data_filtered.loc[input_date_start:input_date_end].index)
    return data_filtered

In [13]:
# def remove_window(input_dataset, input_timestamp_start, input_timestamp_end):
#     """Remove window in data set between specified start and end dates.
    
#     Args:
#         input_dataset (pd.DataFrame): Data indexed and sorted by datetime.
#         input_timestamp_start (Timestamp): The timestamp for which to start removal.
#         input_timestamp_end (Timestamp): The final timestamp to removal.
    
#     Returns:
#         pd.DataFrame sorted and filtered without the specified range.
#     """
#     ## Sort the dataframe
#     # data_subset = input_dataset.sort_index()
#     # Remove the specified time window by dropping indices within the range
#     data_filtered = input_dataset.drop(input_dataset.loc[input_timestamp_start:input_timestamp_end].index)
#     return data_filtered

For large gaps, it may still be useful for the model to keep predictor variables in the time leading up to the useful weir data.

Thus, the rainfall and soil moisture data up to one month prior to any hard cutoff will be kept.

In [14]:
# Simplify data removal
def apply_filter_dates(input_dataset, input_adj = '0 D'):
    """Apply pre-defined date filters.
    
    Args:
        input_dataset (pd.DataFrame): Data indexed by datetime.
        input_adj (str): String argument to account for time leading up to weir data, defaults to none.
    
    Returns:
        pd.DataFrame filtered to the weir range and without the 2-year gap window.
    """
    # data_subset = input_dataset[(date_weir_start - pd.Timedelta(input_adj)):date_weir_end]
    # data_subset = remove_window(input_dataset = data_subset, input_timestamp_start = date_gap_start, input_timestamp_end = (date_gap_end - pd.Timedelta(input_adj)))
    data_adj = filter_dates(
        input_dataset = input_dataset,
        input_date_start = (date_weir_start - pd.Timedelta(input_adj)),
        input_date_end = date_weir_end,
        drop_dates = False
    )
    data_adj = filter_dates(
        input_dataset = data_adj,
        input_date_start = date_gap_start,
        input_date_end = (date_gap_end - pd.Timedelta(input_adj)),
        drop_dates = True
    )
    # data_subset = input_dataset[date_weir_start:date_weir_end]
    # data_subset = remove_window(input_dataset = data_subset, input_timestamp_start = date_gap_start, input_timestamp_end = date_gap_end)
    return data_adj

In [15]:
# Apply filter
gap_adj = '4 W'

data_weir = apply_filter_dates(data_raw_weir)
data_calibration = apply_filter_dates(data_raw_calibration)

# Include data leading up to the weir values and shortly before the end of the 2-yr gap
data_rainfall = apply_filter_dates(data_raw_rainfall, gap_adj)
data_soil_shallow = apply_filter_dates(data_raw_soil_shallow, gap_adj)
data_soil_deep = apply_filter_dates(data_raw_soil_deep, gap_adj)

del gap_adj

In [16]:
# Remove old stuff to save space
del data_raw_calibration, data_raw_weir, data_raw_rainfall, data_raw_soil_shallow, data_raw_soil_deep

### Soil

#### Duplicates

There are some duplicated records between the "shallow" and "deep" data set. Most are identical, but there were two dates with differing records.
It was concluded that those values from the "deep" set with a depth of "0–10" may be eliminated.

In [17]:
# Select the values in the deep data set that have the shallower depth
data_deep_subset = data_soil_deep[data_soil_deep["depth"] != "30-40"]

# Filter set to only be of dates where deep set has shallow values
data_shallow_subset = data_soil_shallow[data_soil_shallow.index.isin(data_deep_subset.index)]

# Inner merge based on date and sample number
data_soil_mismatch = pd.merge(
    # Set the timestamp to be a column, for merging
    data_deep_subset.reset_index(),
    data_shallow_subset.reset_index(),
    # Unite based on timestamp and sample number
    on = ["date", "sample"],
    # Flags
    suffixes = ("_deep", "_shallow"),
    how = "inner"
    )

# Create a variable to indicate if the values match
data_soil_mismatch["match_wet"] = (data_soil_mismatch["h2o_by_wet_deep"] == data_soil_mismatch["h2o_by_wet_shallow"])

## The dry var was not loaded in this analysis, but the exact same issue occurred in it (i.e., the same dates had mismatching values)
# match_all["match_dry"] = (match_all["h2o_by_dry_deep"] == match_all["h2o_by_dry_shallow"])

# Set the sample var to be an integer, for sorting purposes
data_soil_mismatch["sample"] = data_soil_mismatch["sample"].astype('int')

# Sort by date and sample for readability
data_soil_mismatch = data_soil_mismatch.sort_values(by = ['date', 'sample'])

# Remove unneeded columns
data_soil_mismatch = data_soil_mismatch.drop(['chk_fail_shallow', 'chk_fail_deep'], axis=1)

# Filter where there is a mismatch
data_soil_mismatch = data_soil_mismatch[(data_soil_mismatch["match_wet"] == False)]

# Reordering vars for readability
data_soil_mismatch = data_soil_mismatch[['date', 'depth_shallow', 'depth_deep', 'sample', 'h2o_by_wet_shallow', 'h2o_by_wet_deep', 'chk_note_shallow', 'chk_note_deep']]

# Print result
data_soil_mismatch

,date,depth_shallow,depth_deep,sample,h2o_by_wet_shallow,h2o_by_wet_deep,chk_note_shallow,chk_note_deep
0,1989-06-23,10-20,40-50,1,34.700001,35.500000,good,good
1,1989-06-23,1-10,40-50,1,35.400002,35.500000,good,good
27,1989-06-23,1-10,10-20,1,35.400002,34.700001,good,good
42,1989-06-23,10-20,20-30,1,34.700001,36.299999,good,good
43,1989-06-23,1-10,20-30,1,35.400002,36.299999,good,good
19,1989-06-23,1-10,40-50,2,40.400002,36.400002,good,good
29,1989-06-23,1-10,10-20,2,40.400002,36.400002,good,good
44,1989-06-23,10-20,20-30,2,36.400002,35.500000,good,good
45,1989-06-23,1-10,20-30,2,40.400002,35.500000,good,good
2,1989-06-23,10-20,40-50,3,35.900002,35.099998,good,good


In [18]:
# Cleanup
del data_shallow_subset, data_deep_subset, data_soil_mismatch

It was confirmed that values at depth `0-10` in the "deep" data set (including those that do not match the equivalent in the "shallow" set) can be disregarded and excluded from analysis.

In [19]:
# Remove the duplicated samples
data_soil_deep = data_soil_deep[data_soil_deep["depth"] != "0-10"]

#### Abnormal depths

Additional values are reported for a few different, non-standard depths.

In [20]:
print(
    "-----Shallow-----",
    data_soil_shallow.groupby('depth', dropna=False, observed=True)['h2o_by_wet'].count(),
    "\n",
    "-----Deep-----",
    data_soil_deep.groupby('depth', dropna=False, observed=True)['h2o_by_wet'].count(),
    sep="\n"
)

date_abnorm_shallow = data_soil_shallow[data_soil_shallow['depth']!='1-10'].index.unique()
date_abnorm_deep = data_soil_deep[data_soil_deep['depth']!='30-40'].index.unique()

print(
    "\n\nUnique time stamps:",
    "-----Shallow-----",
    date_abnorm_shallow,
    "\n",
    "-----Deep-----",
    date_abnorm_deep,
    sep="\n"
)

# data_soil_shallow.loc['1989-06-23 00:00:00']

-----Shallow-----
depth
1-10     11519
10-20       10
Name: h2o_by_wet, dtype: int64


-----Deep-----
depth
10-20       10
20-30       10
30-40    11519
40-50       10
Name: h2o_by_wet, dtype: int64


Unique time stamps:
-----Shallow-----
DatetimeIndex(['1989-06-23'], dtype='datetime64[ns]', name='date', freq=None)


-----Deep-----
DatetimeIndex(['1989-06-23'], dtype='datetime64[ns]', name='date', freq=None)


Both sets have extra values on the same date. Before removal, it is wise to check that the expected depth samples are present.

In [21]:
print(
    "-----Shallow-----",
    data_soil_shallow.loc[date_abnorm_shallow].groupby('depth', dropna=False, observed=False)['h2o_by_wet'].count(),
    "check:",
    '1-10' in data_soil_shallow.loc[date_abnorm_shallow]['depth'].unique(),
    "\n",
    "-----Deep-----",
    data_soil_deep.loc[date_abnorm_deep].groupby('depth', dropna=False, observed=False)['h2o_by_wet'].count(),
    "check:",
    '30-40' in data_soil_deep.loc[date_abnorm_deep]['depth'].unique(),
    sep="\n"
)

# '1-10' in data_soil_shallow.loc[date_abnorm_shallow]['depth'].unique()

# '30-40' in data_soil_deep.loc[date_abnorm_deep]['depth'].unique()

-----Shallow-----
depth
0-5       0
1-10     10
10-20    10
Name: h2o_by_wet, dtype: int64
check:
True


-----Deep-----
depth
0-10      0
10-20    10
20-30    10
30-40    10
40-50    10
Name: h2o_by_wet, dtype: int64
check:
True


The extra values will be dropped.

In [22]:
# Drop the extra values by selecting only the depths of interest

data_soil_shallow = data_soil_shallow[data_soil_shallow['depth'] == '1-10']

data_soil_deep = data_soil_deep[data_soil_deep['depth'] == '30-40']

In [23]:
# Verify that all abnormal depths have been removed
print(
    data_soil_shallow.groupby('depth', dropna=False, observed=False)['h2o_by_wet'].count(),
    data_soil_deep.groupby('depth', dropna=False, observed=False)['h2o_by_wet'].count()
)

depth
0-5          0
1-10     11519
10-20        0
Name: h2o_by_wet, dtype: int64 depth
0-10         0
10-20        0
20-30        0
30-40    11519
40-50        0
Name: h2o_by_wet, dtype: int64


#### Abnormal locations

Another anomaly occurs with non-standard samples location numbers.

In [24]:
# Get rows where the sample is not the standard 1 through 10
data_soil_sample_abnorm = data_soil_shallow[~data_soil_shallow['sample'].astype('int').isin(list(range(1, 10+1, 1)))].copy()

# Locate other entries from the same date
data_soil_shallow.loc[data_soil_sample_abnorm.index.unique()]

,depth,sample,h2o_by_wet,chk_note,chk_fail
date,,,,,
2005-03-02,1-10,9,29.100000,good,NaN
2005-03-02,1-10,68,39.500000,good,NaN
2005-03-02,1-10,10,29.900000,good,NaN
2005-03-02,1-10,8,36.400002,good,NaN
2005-03-02,1-10,70,43.500000,good,NaN
2005-03-02,1-10,6,40.099998,good,NaN
2005-03-02,1-10,5,41.299999,good,NaN
2005-03-02,1-10,4,35.099998,good,NaN
2005-03-02,1-10,3,38.700001,good,NaN


In [25]:
# Cleanup
del data_soil_sample_abnorm

It is possible that the data entries for samples `1` and `2` were miscoded, however at the moment is is not possible to verify in which way. For now, the values will be dropped.

In [26]:
# Remove non-standard sample locations
data_soil_shallow = data_soil_shallow[data_soil_shallow['sample'].astype('int').isin(list(range(1, 10+1, 1)))]

#### General cleanup

In [27]:
# Remove values flagged as bad or doubtful
data_soil_shallow = data_soil_shallow[
    (data_soil_shallow['chk_note'] != 'bad') &
    (data_soil_shallow['chk_note'] != 'doubtful')
]

data_soil_deep = data_soil_deep[
    (data_soil_deep['chk_note'] != 'bad') &
    (data_soil_deep['chk_note'] != 'doubtful')
]

Remove duplicated entries

In [28]:
# # data_soil_shallow[data_soil_shallow.duplicated(subset=['sample', 'h2o_by_wet'])]
# dss_dup = data_soil_shallow.copy()
# dss_dup = dss_dup.reset_index()
# dss_dup['dup'] = dss_dup.duplicated(subset=['date', 'sample', 'h2o_by_wet'])
# # dss_dup[dss_dup['dup']==True]
# # data_soil_shallow.loc['2023-07-14 00:00:00']

In [29]:
# # Quick copy
# dss_shallow = data_soil_shallow.copy()
# dss_deep = data_soil_deep.copy()

# # Reset indeces to allow for easier filtering
# dss_shallow = dss_shallow.reset_index()
# dss_deep = dss_deep.reset_index()

# dss_shallow = dss_shallow[dss_shallow.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]
# dss_deep = dss_deep[dss_deep.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]

# print(len(dss_shallow), len(dss_deep))

# dss_un = pd.merge(
#     data_soil_shallow.reset_index(),
#     data_soil_deep.reset_index(),
#     on = ["date", "sample"],
#     suffixes = ("_shallow", "_deep"),
#     how = "outer"
#     )

# dss_un = dss_un[dss_un.duplicated(subset=['date', 'sample', 'h2o_by_wet_shallow', 'h2o_by_wet_deep']) == False]
# print(len(dss_un))
# # Return the index
# # united_soil = united_soil.set_index('date')
# # dss_dup = data_soil_shallow.copy()
# # dss_dup = dss_dup.reset_index()
# # dss_dup['dup'] = dss_dup.duplicated(subset=['date', 'sample', 'h2o_by_wet'])

In [30]:
# dss = data_soil_shallow.copy()
# dsd = data_soil_deep.copy()

In [31]:
# # data_water = data_water.reset_index().drop_duplicates(keep='first').set_index('datetime')
# # data_soil_shallow.head()
# ds_test = dss.copy().reset_index()
# ds_test = ds_test.drop_duplicates(subset=['date', 'sample', 'h2o_by_wet'], keep='first')
# print(
#     len(ds_test),
#     len(ds_test[ds_test.duplicated(subset=['date', 'sample']) == False]),
#     len(ds_test.drop_duplicates(subset=['date', 'sample'], keep='first')),
#     len(ds_test.drop_duplicates(subset=['date', 'sample'], keep=False)),
#     sep="\n"
#     )

# # ds_test = ds_test.drop_duplicates(subset=['date', 'sample'], keep=False)
# # ds_test[ds_test.duplicated(subset=['date', 'sample']) == True]
# # ds_test[ds_test['date']=='2023-04-06 00:00:00']

In [32]:
# # Reset indeces to allow for easier filtering
# data_soil_shallow = data_soil_shallow.reset_index()
# data_soil_deep = data_soil_deep.reset_index()
# print("\tshallow", "deep", sep="\t")
# print("Before:\t", len(data_soil_shallow), len(data_soil_deep))

# # Remove duplicate rows
# data_soil_shallow = data_soil_shallow[data_soil_shallow.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]
# data_soil_deep = data_soil_deep[data_soil_deep.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]
# # data_soil_shallow = data_soil_shallow.drop_duplicates(subset=['date', 'sample', 'h2o_by_wet'], keep='first')
# # data_soil_deep = data_soil_deep.drop_duplicates(subset=['date', 'sample', 'h2o_by_wet'], keep='first')
# print("Non-dup:", len(data_soil_shallow), len(data_soil_deep))

# # Remove rows with multiple entries by date and site
# data_soil_shallow = data_soil_shallow[data_soil_shallow.duplicated(subset=['date', 'sample']) == False]
# data_soil_deep = data_soil_deep[data_soil_deep.duplicated(subset=['date', 'sample']) == False]
# # data_soil_shallow = data_soil_shallow.drop_duplicates(subset=['date', 'sample'], keep=False)
# # data_soil_deep = data_soil_deep.drop_duplicates(subset=['date', 'sample'], keep= False)
# print("Singles:", len(data_soil_shallow), len(data_soil_deep))

# # Return indeces
# data_soil_shallow = data_soil_shallow.set_index('date')
# data_soil_deep = data_soil_deep.set_index('date')


In [33]:
# Reset indeces to allow for easier filtering
data_soil_shallow = data_soil_shallow.reset_index()
data_soil_deep = data_soil_deep.reset_index()
print("\tshallow", "deep", sep="\t")
print("Before:\t", len(data_soil_shallow), len(data_soil_deep))

# # Remove duplicate rows
# data_soil_shallow = data_soil_shallow[data_soil_shallow.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]
# data_soil_deep = data_soil_deep[data_soil_deep.duplicated(subset=['date', 'sample', 'h2o_by_wet']) == False]

# Remove duplicate rows
data_soil_shallow = data_soil_shallow.drop_duplicates(subset=['date', 'sample', 'h2o_by_wet'], keep='first')
data_soil_deep = data_soil_deep.drop_duplicates(subset=['date', 'sample', 'h2o_by_wet'], keep='first')
print("Non-dup:", len(data_soil_shallow), len(data_soil_deep))

# # Remove rows with multiple entries by date and site
# data_soil_shallow = data_soil_shallow[data_soil_shallow.duplicated(subset=['date', 'sample']) == False]
# data_soil_deep = data_soil_deep[data_soil_deep.duplicated(subset=['date', 'sample']) == False]

# Remove rows with multiple entries by date and site
data_soil_shallow = data_soil_shallow.drop_duplicates(subset=['date', 'sample'], keep=False)
data_soil_deep = data_soil_deep.drop_duplicates(subset=['date', 'sample'], keep= False)
print("Singles:", len(data_soil_shallow), len(data_soil_deep))

# Return indeces
data_soil_shallow = data_soil_shallow.set_index('date')
data_soil_deep = data_soil_deep.set_index('date')


	shallow	deep
Before:	 11462 11508
Non-dup: 11042 11056
Singles: 10982 11056


In [34]:
data_soil_shallow = data_soil_shallow.drop(columns=['depth', 'chk_note', 'chk_fail'])
data_soil_deep = data_soil_deep.drop(columns=['depth', 'chk_note', 'chk_fail'])

In [35]:
# dss_dup = data_soil_shallow.reset_index()
# dss_dup['dup'] = dss_dup.duplicated(subset=['date', 'sample'])
# dss_dup.set_index('date').loc['2023-04-21 00:00:00']

# data_soil_shallow.reset_index()[data_soil_shallow.reset_index().duplicated(subset=['date', 'sample'])]

### Additional CHART Removals

Only non-`CHART` values will be used for making the model.
Prior to removing them entirely, other missing values must also be dealt with, as they can relate to gaps within CHART-reliant ranges.

In [36]:
# Backup
# data_weir_chart = data_weir.copy()
# data_weir_nochart = data_raw_weir.copy()

# Create a column which will forward fill the source--i.e., fill NAs with the most recent value reported in 'source'
data_weir['source_ffill'] = data_weir['source'].ffill()

# Create a column which will back fill the source--i.e., fill NAs with the next value reported in 'source'
data_weir['source_bfill'] = data_weir['source'].bfill()

# Filtering to remove CHART values and gap fills that rely on CHART values
data_weir = data_weir[
    # Remove CHART values
    (data_weir['source'] != "CHART") &
    # Remove NA values where the most recent source was CHART
    (data_weir['source_ffill'] != "CHART") &
    # Remove NA values where the next source is CHART
    (data_weir['source_bfill'] != "CHART")
]

# Remove extra variables
data_weir = data_weir.drop(['source_ffill', 'source_bfill'], axis=1)

### Missing

Other gaps of missing values occur and should be addressed.
These can be identified by the `chk_note` of 'missing' with a `raw` values of -999.0.
In the manually-adjusted results, the levels of such points have been set to 0.0.
Because there are so few instances of these in the data set, they will simply be removed for this analysis.

*A `chk_note` of 'missing' differs from instances of where a `chk_fail` is a 'Gap Fill'.

In [37]:
# data_weir['2025-07-24 14:00:00':'2025-07-24 17:00:00']

In [38]:
# data_weir[(data_weir['raw'] == -999.0) & (data_weir['chk_note'] == "missing")]
# data_weir_chart['2024-11-15 10:50:00':'2024-11-15 11:15:00']
# data_weir_chart['2024-11-22 11:10:00':'2024-11-22 11:45:00']
# data_weir_chart['2025-07-24 15:10:00':'2025-07-24 16:25:00']

# data_weir[
#     (data_weir['raw'] == -999.0) &
#     (data_weir['chk_note'] == "missing")
# ]
exp_len = len(data_weir[
    (data_weir['raw'] == -999.0) &
    (data_weir['chk_note'] == "missing")
    ]
)

len_before = len(data_weir)

data_weir = data_weir[
    # Invert results
    ~(
        # Selecting rows where raw value is -999.0
        (data_weir['raw'] == -999.0) &
        # and
        # chk_note is missing
        (data_weir['chk_note'] == "missing")
    )
]

if len(data_weir) != (len_before-exp_len):
    print("Check values!")
else:
    print(len_before-len(data_weir), "rows removed.")

del len_before, exp_len

12 rows removed.


### Failure Modes

Some data points have multiple failure modes, and some entries contain duplicate flags.

In [39]:
def fail_bool(input_data, input_string):
    """Return a bool list where the chk_fail column contains a strong.

    Args:
        input_data (DataFrame): Data with a 'chk_fail' column to analyze.
        input_string (str): String to find; case ignored.

    Returns:
        array: List of True/False corresponding to instances of string matches.
    """
    # Return a bool list for an inputted data frame where the chk_fail column contains a string
    # Ignores case, and reports NA values as False
    return np.where(input_data['chk_fail'].str.contains(input_string, case = False, na = False), True, False)

In [40]:
def fail_isolate(input_data):
    """Completes the flagging process for the five major failure modes on an inputted data set.

    Args:
        input_data (DataFrame): Data with a 'chk_fail' column to analyze.

    Returns:
        DataFrame: Data with individual boolean columns of the failure modes.
    """
    # "Obs" is over "Obstruction" used because there are a few instances the flag is truncated.
    input_data['obstruction'] = fail_bool(input_data = input_data, input_string = "Obs")
    input_data['gap_fill'] = fail_bool(input_data = input_data, input_string = "Gap Fill")
    # "Weir" is over "Weir Cleaning" used because there are a few instances the flag is truncated.
    input_data['weir_cleaning'] = fail_bool(input_data = input_data, input_string = "Weir")
    input_data['spike'] = fail_bool(input_data = input_data, input_string = "Spike")
    # "Calib" is over "Calibration" used because there are a few instances the flag is truncated.
    input_data['calibration'] = fail_bool(input_data = input_data, input_string = "Calib")

    # Find all new bool columns
    bool_cols = input_data.select_dtypes(include=['bool']).columns

    # Convert those specific columns to the nullable 'boolean' dtype
    input_data[bool_cols] = input_data[bool_cols].astype('boolean')
    return input_data

In [41]:
data_weir = fail_isolate(data_weir)

data_weir.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3557007 entries, 1989-07-19 11:55:00 to 2025-08-01 13:00:00
Data columns (total 11 columns):
 #   Column         Dtype   
---  ------         -----   
 0   level          float32 
 1   raw            float32 
 2   chk_note       category
 3   chk_fail       category
 4   comment        category
 5   source         category
 6   obstruction    boolean 
 7   gap_fill       boolean 
 8   weir_cleaning  boolean 
 9   spike          boolean 
 10  calibration    boolean 
dtypes: boolean(5), category(4), float32(2)
memory usage: 101.8 MB


## Uniting & Exporting

The calibration, combined (runoff), and rainfall data can be united into a single data frame.
Soil samples do not have the same granularity, so can be stored separately from these so as to avoid duplicated values.

Save the resulting cleaned data frames to parquet files for ease of loading in later analysis.

In [42]:
united_soil = pd.merge(
    data_soil_shallow.reset_index(),
    data_soil_deep.reset_index(),
    on = ["date", "sample"],
    suffixes = ("_shallow", "_deep"),
    how = "outer"
    )

# Return the index
united_soil = united_soil.set_index('date')

# Modifying sample to int for sorting
united_soil["sample"] = united_soil["sample"].astype('int')

# Sorting for readability
united_soil = united_soil.sort_values(by=['date', 'sample'])

# Reset to category
united_soil["sample"] = united_soil["sample"].astype('category')

# Moving sample to front of data frame
soil_samples = united_soil.pop('sample')
united_soil.insert(0, 'sample', soil_samples)
del soil_samples
#

united_soil.info(memory_usage='deep')
united_soil.head()
# Missing values:
# united_soil[united_soil['h2o_by_wet_shallow'].isnull() | united_soil['h2o_by_wet_deep'].isnull()]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11072 entries, 1989-06-23 to 2025-06-26
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   sample              11072 non-null  category
 1   h2o_by_wet_shallow  10982 non-null  float32 
 2   h2o_by_wet_deep     11056 non-null  float32 
dtypes: category(1), float32(2)
memory usage: 184.2 KB


,sample,h2o_by_wet_shallow,h2o_by_wet_deep
date,,,
1989-06-23,1,35.400002,36.099998
1989-06-23,2,40.400002,37.099998
1989-06-23,3,35.299999,35.500000
1989-06-23,4,37.099998,35.099998
1989-06-23,5,41.000000,39.099998


In [43]:
# Export soil data
united_soil.to_parquet(get_data_path('clean/soil.parquet'))

In [44]:
# # Checking column matching
# # Checking to make sure sources match
# def check_cols(input_df, input_col_left, input_col_right, find_mismatch=True):
#     input_df["match"] = (input_df[input_col_left] == input_df[input_col_right]) | (input_df[input_col_left].isnull() & input_df[input_col_right].isnull())
#     if find_mismatch == True:
#         input_df = input_df[(input_df["match"]==False)]
#     return input_df

# check_cols(mini_united, "source_ro", "source_rain")
# check_cols(mini_united, "chk_note_rain", "chk_note_ro")
# check_cols(mini_united, "comment_rain", "comment_ro")

# check_cols(mini_united, 'chk_note_rain', 'chk_note_ro')
# mini_united.dropna(subset="chk_note_rain")
# check_cols(mini_united, 'chk_fail_rain', 'chk_fail_rain')

In [45]:
# # Combine the rainfall and weir data
# united_water_nocal = pd.merge(
#     # Add identifiers to column names
#     data_rainfall.add_suffix("_rain"),
#     data_weir.add_suffix("_ro"),
#     # Merge based on index (datetime)
#     left_index=True,
#     right_index=True,
#     # Keep all data from both frames
#     how='outer'
# )

# united_water_nocal.info(memory_usage='deep')

# # Combine the calibration data with the above merged data
# united_water = pd.merge(
#     # Add identifiers to column names from calibration set
#     data_calibration.add_suffix("_cal"),
#     united_water_nocal,#.copy(),
#     # Merge based on index (datetime)
#     left_index=True,
#     right_index=True,
#     # Keep all data from both frames
#     how='outer'
# )

# united_water_nocal.info(memory_usage='deep')
# united_water.info(memory_usage='deep')

In [46]:
# Combine the rainfall and weir data
united_water_nocal = pd.merge(
    # Add identifiers to column names
    data_rainfall.add_suffix("_rain"),
    data_weir.add_suffix("_ro"),
    # Merge based on index (datetime)
    left_index=True,
    right_index=True,
    # Keep all data from both frames
    how='outer'
)

united_water_nocal.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3581785 entries, 1989-06-21 13:00:00 to 2025-08-01 13:00:00
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   ra_rain           float32 
 1   raw_rain          float32 
 2   chk_note_rain     category
 3   chk_fail_rain     category
 4   level_ro          float32 
 5   raw_ro            float32 
 6   chk_note_ro       category
 7   chk_fail_ro       category
 8   comment_ro        category
 9   source_ro         category
 10  obstruction_ro    boolean 
 11  gap_fill_ro       boolean 
 12  weir_cleaning_ro  boolean 
 13  spike_ro          boolean 
 14  calibration_ro    boolean 
dtypes: boolean(5), category(6), float32(4)
memory usage: 136.6 MB


In [47]:
# Export water data (includes rain and weir info)
united_water_nocal.to_parquet(get_data_path('clean/water_nocal.parquet'))

In [48]:
# Export calibration data (isolated)
data_calibration.to_parquet(get_data_path('clean/calibration.parquet'))

In [49]:
united_water = pd.merge(
    # Add identifiers to column names from calibration set
    data_calibration.add_suffix("_cal"),
    united_water_nocal,#.copy(),
    # Merge based on index (datetime)
    left_index=True,
    right_index=True,
    # Keep all data from both frames
    how='outer'
)

united_water.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3586047 entries, 1989-06-21 13:00:00 to 2025-08-01 13:00:00
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   weir_level_cal    Int8    
 1   ra_rain           float32 
 2   raw_rain          float32 
 3   chk_note_rain     category
 4   chk_fail_rain     category
 5   level_ro          float32 
 6   raw_ro            float32 
 7   chk_note_ro       category
 8   chk_fail_ro       category
 9   comment_ro        category
 10  source_ro         category
 11  obstruction_ro    boolean 
 12  gap_fill_ro       boolean 
 13  weir_cleaning_ro  boolean 
 14  spike_ro          boolean 
 15  calibration_ro    boolean 
dtypes: Int8(1), boolean(5), category(6), float32(4)
memory usage: 143.6 MB


In [50]:
# Export water data (includes rain, weir, AND calibrations)
united_water.to_parquet(get_data_path('clean/water.parquet'))

In [51]:
# pd.merge(
#     # Add identifiers to column names
#     united_water_nocal.memory_usage().to_frame().add_suffix("_nocal"),
#     united_water.memory_usage().to_frame().add_suffix("_cal"),
#     # Merge based on index (datetime)
#     left_index=True,
#     right_index=True,
#     # Keep all data from both frames
#     how='outer'
# )

# # united_water_nocal.memory_usage()

In [52]:
# united_soil['2020-01-01 00:00:00':'2021-12-31 23:59:59']
# united_water['1996-11-01 00:00:00':'1997-01-31 23:59:59']
# 1996-11-05 12:10:00 -- 1997-01-29 08:34:00

In [53]:
# # Water data (not including calibration points)
# united_waternocal = pd.merge(
#     data_rainfall.add_suffix("_rain"), data_weir.add_suffix("_ro"), left_index=True, right_index=True, how='outer')

<!-- ## Export

Save the resulting cleaned data frames to parquet files for ease of loading in later analysis. -->

In [54]:
# print(get_data_path('clean/water.parquet'))
# united_soil.to_parquet(get_data_path('clean/TEST.parquet'))

In [55]:
# # united_water.to_parquet('data/clean/water.parquet')
# # united_soil.to_parquet('data/clean/soil.parquet')

# # Water data (includes rain and weir info)
# united_water_nocal.to_parquet(get_data_path('clean/water_nocal.parquet'))

# # Water data (includes rain, weir, and calibrations)
# united_water.to_parquet(get_data_path('clean/water.parquet'))

# # Soil data
# united_soil.to_parquet(get_data_path('clean/soil.parquet'))

# # Calibration data (isolated)
# data_calibration.to_parquet(get_data_path('clean/calibration.parquet'))

In [56]:
# # data_calibration.to_parquet('data/clean/calibration.parquet')
# # Calibration data (isolated)
# data_calibration.to_parquet(get_data_path('clean/calibration.parquet'))

# united_water_nocal.to_parquet(get_data_path('clean/water_nocal.parquet'))
# # united_waternocal.to_parquet('data/clean/water_nocal.parquet')